# **Pricing Options with Fast Fourier Transform (FFT) — Heston, Variance-Gamma, and Black-Scholes**

This guide demonstrates how to value options using the **Fast Fourier Transform (FFT)**, a powerful mathematical technique for efficient computation. The methods and code presented here are inspired by the course *Financial Engineering and Risk Management* offered by Columbia University through Coursera.

The FFT is particularly useful in financial derivatives pricing due to its ability to handle complex integral formulations quickly and accurately. In this guide, we will:

- Introduce the general concept of FFT and its application in option pricing.
- Explore three pricing models: **Black-Scholes**, **Heston**, and **Variance Gamma**.
- Implement these models using Python’s `numpy` library for FFT calculations.

The FFT technique extends beyond options to other financial instruments like caps, floors, and swaptions, enabling rapid and efficient valuation.

---

## **Foundations of FFT in Option Pricing**

Option pricing via FFT is computationally more efficient than direct numerical integration. Using Python's `numpy.fft.fft` function allows us to leverage optimized computations.

### **Key Rules for FFT Option Pricing**
1. **Log Strike Transformation:** Work in log-space for the strike price, e.g., \( K = 100 \) becomes \( \ln(100) \).
2. **Power of Two:** For maximum efficiency, vectors should have a length of \( 2^n \).
3. **Logarithmic Stock Price Process:** Ensure input values are log-transformed.
4. **Defined Parameters:** Key parameters include:
   - \(\eta\): Step size for integration.
   - \(\alpha\): Damping factor for stability.
   - \(n\): Power for FFT array size (\(2^n\)).
   - \(\beta\): Logarithm of the strike price.


In [ ]:
# Pricing Options using Fast Fourier Transform (FFT)
# This version uses real-life stock data from Yahoo Finance (yfinance)

# Importing Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# Display settings for pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)



In [ ]:
# Fetch real-life stock data using yfinance
def fetch_stock_data(ticker, start_date, end_date):
    """
    Fetch stock data using yfinance and calculate additional metrics.
    
    Parameters:
        ticker (str): Stock ticker symbol (e.g., "AAPL" for Apple).
        start_date (str): Start date for the data (YYYY-MM-DD).
        end_date (str): End date for the data (YYYY-MM-DD).

    Returns:
        pd.DataFrame: DataFrame containing stock price and daily return.
    """
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']]  # Select relevant columns
    stock_data['Daily Return'] = stock_data['Close'].pct_change()  # Calculate daily returns
    stock_data.dropna(inplace=True)
    return stock_data

# Example: Fetch data for Apple (AAPL)
ticker = "AMZN"
start_date = "2024-01-01"
end_date = "2024-12-31"

stock_data = fetch_stock_data(ticker, start_date, end_date)

# Display the stock data
print(f"Stock data for {ticker}:")
stock_data.tail(10)



# **The Models**

---

### **1. Black-Scholes Model**

The **Black-Scholes model** is foundational in finance, assuming a log-normal distribution of asset prices. It models the option price using a deterministic volatility parameter, solving a partial differential equation analogous to the heat equation in physics.

#### **Key Features:**

- **Efficient Markets:** Assumes no arbitrage and continuous trading.
- **Constant Volatility:** Simplifies calculations but fails to capture market dynamics like volatility smiles.
- **Risk-Free Interest Rate:** Assumes a constant rate throughout the option's life.

The Black-Scholes process is defined by:

\[
\frac{\partial V}{\partial t} + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} + r S \frac{\partial V}{\partial S} - r V = 0
\]

Its characteristic function is required for FFT implementation.

---

### **2. Heston Model**

The **Heston model** improves upon Black-Scholes by introducing stochastic volatility, capturing market phenomena like volatility smiles and mean-reverting behavior.

#### **Key Features:**

- **Stochastic Volatility:** Variance follows a mean-reverting process:  
  \[
  dv_t = \kappa (\theta - v_t) dt + \eta \sqrt{v_t} dW_t
  \]
- **Mean-Reversion:** Volatility tends toward a long-term average.
- **Correlation Parameter (\(\rho\)):** Allows coupling between asset price and volatility dynamics.

The Heston model's flexibility makes it a popular choice for capturing real-world market behaviors.

---

### **3. Variance-Gamma (VG) Model**

The **Variance Gamma model** introduces jumps and stochastic volatility, addressing the shortcomings of simpler models.

#### **Key Features:**

- **Jumps in Prices:** Captures sudden, large asset price movements.
- **Flexibility:** Models skewness (asymmetry) and kurtosis (fat tails).
- **Convergence:** Reduces to Black-Scholes for specific parameter settings.

The VG model's characteristic function provides the basis for FFT computations in this context.



In [ ]:
# General Parameters for Option Pricing
S0 = float(stock_data['Close'].iloc[-1])  # Convert to scalar (float)
K = S0 * 1.05                            # Strike price (5% above current stock price)
k = np.log(K)                            # Log strike price
r = 0.03                                 # Risk-free interest rate
q = 0.01                                 # Dividend yield
T = 0.5                                  # Time to maturity (6 months)

# FFT Parameters
n = 12                                   # Exponent for 2^n
N = 2 ** n                               # Number of steps
eta = 0.25                               # Integration step size
alpha = 1.0                              # Damping factor
lambda_ = (2 * np.pi / N) / eta          # Log-strike step size
beta = np.log(K)                         # Log-strike value

# Display Parameters
print(f"Option Pricing Parameters for {ticker}:")
pd.DataFrame({
    'Parameter': ['Initial Stock Price (S0)', 'Strike Price (K)', 'Risk-free Rate (r)', 
                  'Dividend Yield (q)', 'Time to Maturity (T)', 'FFT Steps (N)', 
                  'Integration Step Size (eta)', 'Damping Factor (alpha)'],
    'Value': [S0, K, r, q, T, N, eta, alpha]
})



In [ ]:
# Function to Compute Characteristic Function
def compute_characteristic_function(u, params, S0, r, q, T, model):
    """
    Calculate the characteristic function for different option pricing models.

    Parameters:
        u (array): Complex inputs for the Fourier transform.
        params (list): Model-specific parameters.
        S0 (float): Current stock price.
        r (float): Risk-free interest rate.
        q (float): Dividend yield.
        T (float): Time to maturity.
        model (str): Option pricing model ("BS" for Black-Scholes).

    Returns:
        np.ndarray: Values of the characteristic function.
    """
    if model == 'BS':  # Black-Scholes
        volatility = params[0]
        drift = np.log(S0) + (r - q - 0.5 * volatility**2) * T
        diffusion = volatility * np.sqrt(T)
        phi = np.exp(1j * drift * u - 0.5 * (diffusion * u)**2)
    elif model == 'Heston':  # Heston
        kappa, theta, sigma, rho, v0 = params
        temp1 = kappa - 1j * rho * sigma * u
        g = np.sqrt(sigma**2 * (u**2 + 1j * u) + temp1**2)
        power_term = 2 * kappa * theta / sigma**2
        log_numerator = np.log(np.cosh(g * T / 2) + (temp1 / g) * np.sinh(g * T / 2))
        log_denominator = (u**2 + 1j * u) * v0 / (g / np.tanh(g * T / 2) + temp1)
        phi = np.exp(1j * u * np.log(S0) + r * T * 1j * u - log_numerator - log_denominator)
    elif model == 'VG':  # Variance-Gamma
        sigma, nu, theta = params
        if nu == 0:
            mu = np.log(S0) + (r - q - theta - 0.5 * sigma**2) * T
            phi = np.exp(1j * u * mu) * np.exp((1j * theta * u - 0.5 * sigma**2 * u**2) * T)
        else:
            mu = np.log(S0) + (r - q + np.log(1 - theta * nu - 0.5 * sigma**2 * nu) / nu) * T
            phi = np.exp(1j * u * mu) * ((1 - 1j * nu * theta * u + 0.5 * nu * sigma**2 * u**2)**(-T / nu))
    return phi



In [ ]:
# Pricing Options for Black-Scholes, Heston, and Variance-Gamma Models
def price_options_and_plot(params, S0, K, r, q, T, alpha, eta, n, model, title):
    km_values, cT_km_values = calculate_fft_values(params, S0, K, r, q, T, alpha, eta, n, model)
    
    # Convert Log-Strike to Normal Strike Prices
    strikes = np.exp(km_values)
    
    # Create a DataFrame for Results
    results = pd.DataFrame({'Strike Price': strikes, 'Option Price': cT_km_values})
    
    # Plotting the Results
    #plt.figure(figsize=(10, 6))
    #plt.plot(strikes, cT_km_values, label=f"{model} Option Prices", color='blue')
    #plt.axvline(x=K, color='red', linestyle='--', label='Strike Price (K)')
    #plt.title(f"{title} Option Prices using FFT")
    #plt.xlabel("Strike Price")
    #plt.ylabel("Option Price")
    #plt.legend()
    #plt.grid(True)
    #plt.show()
    
    return results



In [ ]:
# Fetch Option Chain using yfinance
def fetch_option_chain(ticker, expiration_date):
    """
    Fetch option chain for a given ticker and expiration date.

    Parameters:
        ticker (str): Stock ticker symbol.
        expiration_date (str): Expiration date (YYYY-MM-DD).

    Returns:
        pd.DataFrame: Option chain data.
    """
    stock = yf.Ticker(ticker)
    option_chain = stock.option_chain(expiration_date)
    calls = option_chain.calls
    return calls

# Example: Fetch option chain for AAPL
expiration_date = "2025-01-03"  # Example expiration date
option_chain = fetch_option_chain(ticker, expiration_date)

# Display Option Chain
option_chain[['strike', 'lastPrice', 'impliedVolatility', 'volume']].head()

# Black-Scholes Parameters
model = 'BS'
volatility = stock_data['Daily Return'].std() * np.sqrt(252)  # Annualized volatility
params = [volatility]

print(f"Pricing options for {ticker} using Black-Scholes Model:")
bs_results = price_options_and_plot(params, S0, K, r, q, T, alpha, eta, n, model, "Black-Scholes")
print(bs_results.head())

